In [1]:
import csv, json, re
from pandas.io.json._table_schema import build_table_schema
import requests
from vnstock import *

In [2]:
def write_json_schema(df, file):
    schema = build_table_schema(df, index=False)
    schema = schema['fields']
    schema = str(schema)
    schema = re.sub("\[", "", schema)
    schema = re.sub("\]", "", schema)
    schema = re.sub("'", "\"", schema)
    schema = re.sub("{\"name\": ", "", schema)
    schema = re.sub(", \"type\"", ": {\"type\"", schema)


    header = """
    {
        "type": "object",
        "description": "",
        "properties": {
    """

    ending = """
        }
    }
    """
    schema = header + schema + ending
    f = open(file, "w")
    f.write(schema)
    f.close()

    return schema

In [5]:
generalrating = general_rating("TCB")
write_json_schema(valuation_rating("TCB"), "source-tcbs-valuation-rating/source_tcbs_valuation_rating/schemas/valuation_rating.json")

'\n    {\n        "type": "object",\n        "description": "",\n        "properties": {\n    "industryEn": {"type": "string"}, "ticker": {"type": "string"}, "valuation": {"type": "number"}, "pe": {"type": "integer"}, "pb": {"type": "integer"}, "ps": {"type": "string"}, "evebitda": {"type": "string"}, "dividendRate": {"type": "integer"}\n        }\n    }\n    '

In [ ]:
def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            if len(row['ticker']) < 4:
                jsonArray.append(row['ticker'])
    
    jsonArray = re.sub(' ', '', str(jsonArray)) 
    jsonArray = re.sub('\'', '', str(jsonArray))
    jsonArray = jsonArray[1:-1]
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as f: 
        f.write(jsonArray)

In [ ]:
csvFilePath = r'/home/jazzdung/Library/Downloads/vn_stock_listing_companies_2023-05-20.csv'
jsonFilePath = r'symbol.txt'
org = csv_to_json(csvFilePath, jsonFilePath)

In [ ]:
def write_configured_catalog(df_list, file):
    final_json = """
    {
    "streams": [
    """

    for df in df_list:
        schema = build_table_schema(df)
        schema = schema['fields']
        schema = str(schema[1:])
        schema = re.sub("\[", "", schema)
        schema = re.sub("\]", "", schema)
        schema = re.sub("'", "\"", schema)
        schema = re.sub("{\"name\": ", "", schema)
        schema = re.sub(", \"type\"", ": {\"type\"", schema)


        header = """
        {
            "stream": {
            "name": "balance_sheet",
            "description": "Stock data price history",
            "json_schema": {
                "$schema": "http://json-schema.org/draft-04/schema#",
        """

        ending = """
            },
            "supported_sync_modes": ["full_refresh"]
            },
            "sync_mode": "full_refresh",
            "destination_sync_mode": "append"
        }
        """

        schema = header + schema + ending

        final_json += schema
    
    final_json += "] }"
    f = open(file, "w")
    f.write(final_json)
    f.close()

    return schema

In [ ]:
balancesheet = financial_flow('TCB', 'balancesheet','yearly')
incomestatement = financial_flow('TCB', 'incomestatement','yearly')
cashflow = financial_flow('TCB', 'cashflow','yearly')

write_configured_catalog([generalrating], "integration_tests/configured_catalog.json")

In [9]:

url = f'https://apipubaws.tcbs.com.vn/tcanalysis/v1/finance/TCB/incomestatement?yearly=1&isAll=true'

response = requests.get(url).json()

for record in response:
    print(record)

{'ticker': 'TCB', 'quarter': 5, 'year': 2022, 'revenue': 30290, 'yearRevenueGrowth': 0.135, 'quarterRevenueGrowth': None, 'costOfGoodSold': None, 'grossProfit': None, 'operationExpense': -13398, 'operationProfit': 40902, 'yearOperationProfitGrowth': 0.103, 'quarterOperationProfitGrowth': None, 'interestExpense': None, 'preTaxProfit': 25568, 'postTaxProfit': 20436, 'shareHolderIncome': 20150, 'yearShareHolderIncomeGrowth': 0.116, 'quarterShareHolderIncomeGrowth': None, 'investProfit': -83, 'serviceProfit': 8527, 'otherProfit': 2167, 'provisionExpense': -1936, 'operationIncome': 27504, 'ebitda': None}
{'ticker': 'TCB', 'quarter': 5, 'year': 2021, 'revenue': 26699, 'yearRevenueGrowth': 0.424, 'quarterRevenueGrowth': None, 'costOfGoodSold': None, 'grossProfit': None, 'operationExpense': -11173, 'operationProfit': 37076, 'yearOperationProfitGrowth': 0.371, 'quarterOperationProfitGrowth': None, 'interestExpense': None, 'preTaxProfit': 23238, 'postTaxProfit': 18415, 'shareHolderIncome': 18052